In [ ]:
###########################################################
# MSDS 442: AI Agent Design and Development
# Spring '25
# Dr. Bader
#
# Assignment 4 - Northwestern Memorial – Healthcare Agent
# 
# Kevin Geidel
#
###########################################################

# OBJECTIVE:
#   The following will construct multiple AI agents using the LangChain & LangGraph frameworks. 
#   The agents will represent different departments of Northwestern Memorial Hospital.
#   They will coordinate, synchronize, and act to answer patients'/visitors' questions.

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Python native imports
import os, textwrap, json

# 3rd party package imports
from IPython.display import display, Image
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Assign experiment-wide variables
model_name = 'gpt-4o-mini'
data_dir = os.path.join('reports', 'Assignment_4')
knowledge_base_dir = os.path.join('knowledge_base')

In [ ]:
# Requirement 1: Define the structure of agent state for the LangGraph
class InquiryState(TypedDict):
    inquiry: str
    next_node: str
    response: str

In [ ]:
# Establish the AI client
llm = ChatOpenAI(model=model_name, temperature=0) 

In [ ]:
# Define utils needed by the agents

def load_knowledge_base(filename):
    # Extract inquires and responses from the JSON format knowledge base
    full_path = os.path.join(knowledge_base_dir, filename)
    with open(full_path, 'r') as file:
        data = json.load(file)
    return str(data)

def get_query_from_inquiry(inquiry):
    return f"""Provide an answer for following user's inquiry: '{inquiry}' using the knowledge_base."""

def get_human_message_for_agent(state):
    # Return the "HumanMessage" that forwards the user's inquiry (or last agent's inquiry) to the next agent
    return HumanMessage(
            content=[
                {"type": "text", "text": get_query_from_inquiry(state['inquiry'])},
            ],
        )

def get_system_message_for_agent(knowledge_base_filename):
    return SystemMessage(
        content=f"You are a helpful assistant tasked with answering user's inquiry based on the answers you have in this knowledge_base only: {load_knowledge_base(knowledge_base_filename)}"
    )

In [ ]:
def operator_router(state):    
    query = f"""Classify the user's intents based on the following input: '{state['inquiry']}'. 
            List of possible intent values: Greeting, GeneralInquiry, ER, Radiology, PrimaryCare, Cardiology, Pediatrics, BillingInsurance
            Return only the intent value of the inquiry identified with no extra text or characters"""
    
    human_message = HumanMessage(
        content=[
            {"type": "text", "text": query},
        ],
    )

    system_message = SystemMessage(content="You are a helpful assistant tasked with classifying the intent of user's inquiry")
    
    response = llm.invoke([system_message]+[human_message])
    intent = response.content.strip()
            
    response_lower = intent.lower()
    
    if "greeting" in response_lower:
        response = "Hello there, This is Northwestern Memorial Hospital, How can I assist you today?"
        next_node = END
    elif "generalinquiry" in response_lower:
        response = "For general informtion about nearby parking, hotels and restaurants, please visit https://www.nm.org/ and navigate to Patients & Visitors link "
        next_node = END
    else:
        response = None
        next_node = intent

    return {
        "inquiry": state["inquiry"],
        "next_node": next_node,
        "response": response
    }


In [ ]:
def er_agent(state):
    print("\n\n ER KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"inquiry": state["inquiry"], "next_node": END, "response": "ER: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE"}

In [ ]:
def radiology_agent(state):
    print("\n\n Radiology KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"inquiry": state["inquiry"], "next_node": END, "response": "Radiology: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE"}

In [ ]:
def primary_care_agent(state):
    print("\n\n Primary Care KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"inquiry": state["inquiry"], "next_node": END, "response": "Primary Care: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE"}

In [ ]:
def cardiology_agent(state):
    # Handle inquires related to the Cardiology department
    response = llm.invoke([get_system_message_for_agent('cardiology.json')]+[get_human_message_for_agent(state)])
    formatted_response = "Cardiology:: " + response.content.strip()
    return {"input": state["inquiry"], "next_node": END, "response": formatted_response}

In [ ]:
def pediatrics_agent(state):
    print("\n\n Pediatrics KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"input": state["inquiry"], "next_node": END, "response": "Pediatrics: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE."}


In [ ]:
def billing_agent(state):
    print("\n\n BillingInsurance KNOWLEDGE-BASE IS EMPTY \n\n ")
    return {"input": state["inquiry"], "next_node": END, "response": "BillingInsurance: YOU NEED TO ADD-YOUR-KNOWLEDGE-BASE"}
    

In [ ]:

builder = StateGraph(InquiryState)

builder.add_node("Operator", operator_router)
builder.add_node("ER", er_agent)
builder.add_node("Radiology", radiology_agent)
builder.add_node("PrimaryCare", primary_care_agent)
builder.add_node("Cardiology", cardiology_agent)
builder.add_node("Pediatrics", pediatrics_agent)
builder.add_node("BillingInsurance", billing_agent)

builder.set_entry_point("Operator")

builder.add_conditional_edges(
    "Operator",
    lambda x: x["next_node"],
    {
        "ER": "ER",
        "PrimaryCare": "PrimaryCare",
        "Pediatrics": "Pediatrics",
        "Radiology": "Radiology",
        "Cardiology": "Cardiology",
        "BillingInsurance": "BillingInsurance",
        END: END
    }
)

for node in ["ER", "Radiology", "PrimaryCare", "Cardiology", "Pediatrics", "BillingInsurance"]:
    builder.add_edge(node, END)


graph = builder.compile()


In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
while True:
    user_input = input("User: ")
    print(f"\nUser:\n\n {user_input}")
    if user_input.lower() in {"q", "quit"}:
        print("Goodbye!")
        break
    result = graph.invoke({"inquiry": user_input})
    
    response = result.get("response", "No Response Returned")
    print(f"\n\nResponse:\n\n {textwrap.fill(response, width=100)} \n\n")